<a href="https://colab.research.google.com/github/CodeHunterOfficial/AI_DataMining/blob/main/%D0%A1%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0/t_SNE_(t_distributed_Stochastic_Neighbor_Embedding).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## t-SNE (t-distributed Stochastic Neighbor Embedding)

### Введение

t-SNE (t-distributed Stochastic Neighbor Embedding) — это метод нелинейного снижения размерности, предложенный Лауренсом ван дер Маатеном и Джеффри Хинтоном в 2008 году. Основная цель t-SNE — визуализация высокоразмерных данных, таких как изображения, текст, геномные данные, в двумерном или трехмерном пространстве, при этом сохраняя структуру данных и делая возможной интерпретацию в малом измерении.

Применение t-SNE особенно полезно для данных, где линейные методы, такие как PCA (Principal Component Analysis), недостаточно хорошо сохраняют структуру данных, так как t-SNE лучше отображает локальные зависимости и кластеры.

### Общая идея t-SNE

Алгоритм t-SNE переводит высокоразмерные данные в низкоразмерное пространство, сохраняя относительные расстояния между точками. Основной подход t-SNE заключается в том, чтобы превратить отношения между точками в высокоразмерном пространстве в вероятности и попытаться сохранить эти вероятности при проекции точек в низкоразмерное пространство.

Метод t-SNE пытается минимизировать расхождение между двумя распределениями:
1. **Вероятностное распределение в исходном высокоразмерном пространстве.**
2. **Вероятностное распределение в целевом низкоразмерном пространстве.**

Этот процесс минимизации разности вероятностей делает t-SNE способным сохранять локальные структуры данных.

### Построение вероятностного распределения в высокоразмерном пространстве

#### Сходство между точками в исходном пространстве

В исходном пространстве данные могут иметь большое количество признаков, и для каждого объекта $x_i$ мы стремимся определить, какие объекты находятся "близко". Это делается путём вычисления **вероятности того, что объект $x_j$ является соседом объекта $x_i$**, используя Гауссово ядро (функцию Гаусса). Вероятность, что точка $x_j$ является соседом точки $x_i$, вычисляется по следующей формуле:

$$
p_{j|i} = \frac{\exp\left(-\frac{||x_i - x_j||^2}{2\sigma_i^2}\right)}{\sum_{k \neq i} \exp\left(-\frac{||x_i - x_k||^2}{2\sigma_i^2}\right)}
$$

Где:
- $||x_i - x_j||^2$ — квадрат евклидова расстояния между точками $x_i$ и $x_j$,
- $\sigma_i$ — параметр сглаживания (ширина Гауссова ядра), который контролирует, насколько широко рассматриваются соседние точки для точки $x_i$.

Эта вероятность $p_{j|i}$ показывает вероятность того, что точка $j$ находится близко к $i$ в исходном пространстве. Однако, чтобы создать симметричное распределение вероятностей для пары точек $i$ и $j$, мы вычисляем:

$$
p_{ij} = \frac{p_{j|i} + p_{i|j}}{2n}
$$

Где:
- $n$ — общее число точек в наборе данных.

Таким образом, $p_{ij}$ отражает симметричную вероятность того, что точки $i$ и $j$ являются соседями в исходном пространстве.

### Построение вероятностного распределения в низкоразмерном пространстве

#### Сходство между точками в низкоразмерном пространстве

После проекции данных в низкоразмерное пространство $y$, необходимо аналогичным образом вычислить вероятности соседства между точками в этом пространстве. Однако, вместо Гауссова распределения t-SNE использует **распределение Стьюдента с одной степенью свободы (распределение Коши)** для моделирования сходства точек. Вероятность $q_{ij}$, что точки $y_i$ и $y_j$ являются соседями в низкоразмерном пространстве, задаётся как:

$$
q_{ij} = \frac{\left(1 + ||y_i - y_j||^2\right)^{-1}}{\sum_{k \neq l} \left(1 + ||y_k - y_l||^2\right)^{-1}}
$$

Здесь:
- $||y_i - y_j||^2$ — квадрат евклидова расстояния между точками $y_i$ и $y_j$ в низкоразмерном пространстве.

Использование распределения Стьюдента с одной степенью свободы вместо Гауссова распределения связано с его "плоским" хвостом, который уменьшает влияние больших расстояний. Это помогает точкам, находящимся далеко друг от друга в исходном пространстве, не "сближаться" в проекции, что делает t-SNE устойчивым к выбросам и хорошо подходящим для кластеризации.

### Минимизация расхождения между распределениями

Чтобы обеспечить, что проекция сохраняет структуру исходных данных, t-SNE минимизирует расхождение между двумя распределениями $P$ (в исходном пространстве) и $Q$ (в низкоразмерном пространстве) с помощью **дивергенции Кульбака-Лейблера (KL-дивергенции)**. KL-дивергенция измеряет разницу между двумя вероятностными распределениями:

$$
D_{KL}(P || Q) = \sum_{i} \sum_{j} p_{ij} \log \frac{p_{ij}}{q_{ij}}
$$

Функция стоимости t-SNE:

$$
C = \sum_{i} \sum_{j} p_{ij} \log \frac{p_{ij}}{q_{ij}}
$$

Здесь:
- $p_{ij}$ — вероятность в исходном пространстве,
- $q_{ij}$ — вероятность в целевом пространстве.

Минимизация этой функции стоимости означает, что t-SNE будет настраивать координаты точек $y_i$ в низкоразмерном пространстве так, чтобы распределения $p_{ij}$ и $q_{ij}$ были как можно более похожими. Основной метод, используемый для минимизации этой функции, — стохастический градиентный спуск (SGD).

#### Градиент функции стоимости

Градиенты функции стоимости $C$ по координатам точек $y_i$ вычисляются следующим образом:

$$
\frac{\partial C}{\partial y_i} = 4 \sum_j (p_{ij} - q_{ij}) (y_i - y_j) \left(1 + ||y_i - y_j||^2\right)^{-1}
$$

Этот градиент показывает, как необходимо корректировать положение каждой точки $y_i$ в низкоразмерном пространстве, чтобы минимизировать расхождение между распределениями.

Если $p_{ij} > q_{ij}$ (то есть точки ближе друг к другу в исходном пространстве, чем в низкоразмерном), t-SNE сдвинет точки ближе. Если наоборот, $q_{ij} > p_{ij}$, t-SNE будет стремиться увеличить расстояние между точками.

### Примеры применения

#### Пример 1: Визуализация рукописных цифр (MNIST)

t-SNE часто используется для визуализации данных рукописных цифр MNIST, состоящих из 60 000 изображений размером $28 \times 28$ пикселей. Размерность каждого изображения векторизуется в 784 признака, что делает задачу визуализации сложной. t-SNE помогает отобразить эти изображения в двумерное пространство, при этом цифры с похожими чертами группируются вместе (например, "0" и "8" могут образовывать отдельные кластеры).

#### Пример 2: Анализ генетических данных

t-SNE широко применяется в биоинформатике для анализа данных экспрессии генов. Когда данные имеют тысячи признаков, t-SNE помогает визуализировать взаимосвязи между образцами, выделяя кластеры, соответствующие различным биологическим состояниям или группам заболеваний.

#### Пример 3: Визуализация словарных векторов

Алгоритмы, такие как word2vec и GloVe, обучают векторные представления слов, где семантически похожие слова находятся рядом друг с другом в векторном пространстве. Однако такие представления обычно имеют высокую размерность (например, 300). t-SNE помогает визуализировать эти высокоразмерные векторы слов, отображая их в двумерное пространство, где слова, имеющие схожее значение, будут находиться рядом (например, "король" и "королева").










Давайте рассмотрим конкретный пример применения t-SNE, включая все математические расчеты, чтобы проиллюстрировать, как работает метод. Будем использовать упрощенный набор данных для наглядности и пройдем по всем этапам работы алгоритма.

### Пример: Применение t-SNE на 3 точках в двумерном пространстве

Предположим, у нас есть три точки в двумерном пространстве:

$$
\mathbf{x}_1 = (0, 0), \quad \mathbf{x}_2 = (1, 0), \quad \mathbf{x}_3 = (0, 1)
$$

Наша задача — визуализировать эти точки в одномерном пространстве с использованием алгоритма t-SNE. Мы пройдем через все шаги: вычисление расстояний, вероятностей, минимизацию KL-дивергенции и коррекцию координат.

### Шаг 1: Вычисление расстояний между точками в исходном пространстве

Первое, что нам нужно сделать, — это вычислить евклидовы расстояния между всеми парами точек в исходном пространстве.

$$
d_{12} = ||\mathbf{x}_1 - \mathbf{x}_2|| = \sqrt{(0 - 1)^2 + (0 - 0)^2} = 1
$$
$$
d_{13} = ||\mathbf{x}_1 - \mathbf{x}_3|| = \sqrt{(0 - 0)^2 + (0 - 1)^2} = 1
$$
$$
d_{23} = ||\mathbf{x}_2 - \mathbf{x}_3|| = \sqrt{(1 - 0)^2 + (0 - 1)^2} = \sqrt{2} \approx 1.414
$$

### Шаг 2: Вычисление условных вероятностей $p_{ij}$ в исходном пространстве

Теперь для каждой пары точек $(\mathbf{x}_i, \mathbf{x}_j)$ нужно вычислить вероятность $p_{ij}$, которая отражает "близость" этих точек в исходном пространстве. Вероятность рассчитывается на основе гауссового распределения. Формула для $p_{ij}$:

$$
p_{ij} = \frac{\exp\left(-d_{ij}^2 / 2 \sigma^2\right)}{\sum_{k \neq i} \exp\left(-d_{ik}^2 / 2 \sigma^2\right)}
$$

Допустим, возьмем $\sigma^2 = 1$ для простоты.

Для пары $\mathbf{x}_1$ и $\mathbf{x}_2$:

$$
p_{12} = \frac{\exp\left(-1^2 / 2\right)}{\exp\left(-1^2 / 2\right) + \exp\left(-1^2 / 2\right)} = \frac{\exp(-0.5)}{2 \exp(-0.5)} = \frac{1}{2}
$$

Для пары $\mathbf{x}_1$ и $\mathbf{x}_3$:

$$
p_{13} = \frac{\exp\left(-1^2 / 2\right)}{2 \exp(-0.5)} = \frac{1}{2}
$$

Аналогично для пары $\mathbf{x}_2$ и $\mathbf{x}_3$:

$$
p_{23} = \frac{\exp\left(-(\sqrt{2})^2 / 2\right)}{\exp\left(-1^2 / 2\right) + \exp\left(-(\sqrt{2})^2 / 2\right)} = \frac{\exp(-1)}{\exp(-0.5) + \exp(-1)} \approx \frac{0.3679}{0.6065 + 0.3679} = \frac{0.3679}{0.9744} \approx 0.3776
$$

Для $p_{21}$ и $p_{31}$, как и для $p_{ij}$, значения симметричны, поэтому они равны.

Итак, матрица вероятностей $p$ в исходном пространстве:

$$
p = \begin{bmatrix}
0 & 0.5 & 0.5 \\
0.5 & 0 & 0.3776 \\
0.5 & 0.3776 & 0
\end{bmatrix}
$$

### Шаг 3: Проекция точек в низкоразмерное пространство

Предположим, что начальные позиции точек в одномерном пространстве $y_1, y_2, y_3$ равны:

$$
y_1 = 0, \quad y_2 = 0.5, \quad y_3 = 1
$$

Теперь нужно вычислить расстояния между точками в новом пространстве $y_i$:

$$
||y_1 - y_2||^2 = (0 - 0.5)^2 = 0.25
$$
$$
||y_1 - y_3||^2 = (0 - 1)^2 = 1
$$
$$
||y_2 - y_3||^2 = (0.5 - 1)^2 = 0.25
$$

### Шаг 4: Вычисление вероятностей $q_{ij}$ в низкоразмерном пространстве

Вероятности $q_{ij}$ вычисляются с использованием распределения Стьюдента с одной степенью свободы. Формула для $q_{ij}$:

$$
q_{ij} = \frac{\left(1 + ||y_i - y_j||^2\right)^{-1}}{\sum_{k \neq l} \left(1 + ||y_k - y_l||^2\right)^{-1}}
$$

Для $q_{12}$:

$$
q_{12} = \frac{(1 + 0.25)^{-1}}{(1 + 0.25)^{-1} + (1 + 1)^{-1} + (1 + 0.25)^{-1}} = \frac{1/1.25}{1/1.25 + 1/1 + 1/1.25} = \frac{0.8}{0.8 + 1 + 0.8} = \frac{0.8}{2.6} \approx 0.3077
$$

Для $q_{13}$:

$$
q_{13} = \frac{1/2}{2.6} \approx 0.3846
$$

Для $q_{23}$:

$$
q_{23} = \frac{1/1.25}{2.6} \approx 0.3077
$$

Итак, матрица вероятностей $q$ в низкоразмерном пространстве:

$$
q = \begin{bmatrix}
0 & 0.3077 & 0.3846 \\
0.3077 & 0 & 0.3077 \\
0.3846 & 0.3077 & 0
\end{bmatrix}
$$

### Шаг 5: Минимизация KL-дивергенции

Цель t-SNE — минимизировать расхождение между распределениями $p$ и $q$. KL-дивергенция между $p_{ij}$ и $q_{ij}$ вычисляется по формуле:

$$
C = \sum_{i} \sum_{j} p_{ij} \log \frac{p_{ij}}{q_{ij}}
$$

Для пары $p_{12}$ и $q_{12}$:

$$
C_{12} = p_{12} \log \frac{p_{12}}{q_{12}} = 0.5 \log \frac{0.5}{0.3077} = 0.5 \log 1.625 \approx 0.5 \times 0.2107 = 0.1054
$$

Для пары $p_{13}$ и $q_{13}$:

$$
C_{13} = 0.5 \log \frac{0.5}{0.3846} = 0.5 \log 1.3 \approx 0.5 \times 0.1139 = 0.0569
$$

Для пары $p_{23}$ и $q_{23}$:

$$
C_{23} = 0.3776 \log \frac{0.3776}{0.3077} = 0.3776 \log 1.227 \approx 0.3776 \times 0.0919 = 0.0347
$$

Общая KL-дивергенция:

$$
C = 0.1054 + 0.0569 + 0.0347 = 0.197
$$






Продолжаем решение задачи t-SNE, останавливаясь на этапе минимизации KL-дивергенции и обновления координат. Мы разобрали, как вычисляются вероятности $p_{ij}$ и $q_{ij}$, а также рассчитали KL-дивергенцию. Теперь нам нужно обновить координаты точек в низкоразмерном пространстве для минимизации этой дивергенции.

### Шаг 6: Обновление координат с помощью градиентного спуска

Основная цель t-SNE — минимизировать расхождение между распределениями $p_{ij}$ (в высокоразмерном пространстве) и $q_{ij}$ (в низкоразмерном пространстве). Это достигается с помощью оптимизации функции стоимости, которая является KL-дивергенцией между двумя распределениями:

$$
C = \sum_{i} \sum_{j} p_{ij} \log \frac{p_{ij}}{q_{ij}}
$$

Чтобы минимизировать эту функцию, используется градиентный спуск. Мы будем изменять координаты точек в новом пространстве, стремясь уменьшить значение $C$.

#### Градиент функции стоимости

Градиент функции стоимости $C$ по координатам $y_i$ вычисляется следующим образом:

$$
\frac{\partial C}{\partial y_i} = 4 \sum_j \left( (p_{ij} - q_{ij}) q_{ij} (y_i - y_j) \right)
$$

Здесь $y_i$ — координаты точки $i$ в новом пространстве, $p_{ij}$ и $q_{ij}$ — вероятности, соответствующие точкам $i$ и $j$ в исходном и проекционном пространствах соответственно.

Градиент показывает, в каком направлении нужно двигать точку $y_i$, чтобы уменьшить стоимость $C$. Давайте подробно рассмотрим, как этот градиент используется для обновления координат.

### Шаг 7: Итеративное обновление координат

Алгоритм t-SNE на каждом шаге итеративно корректирует координаты точек в новом пространстве, применяя следующий шаг градиентного спуска:

$$
y_i^{(new)} = y_i^{(old)} - \eta \frac{\partial C}{\partial y_i}
$$

Здесь $\eta$ — коэффициент обучения, который определяет скорость изменения координат.

#### Вычисление нового положения для каждой точки:

1. Для точки $y_1$:
   
   $$
   \frac{\partial C}{\partial y_1} = 4 \left( (p_{12} - q_{12}) q_{12} (y_1 - y_2) + (p_{13} - q_{13}) q_{13} (y_1 - y_3) \right)
   $$
   
   Подставим значения $p_{12} = 0.5$, $q_{12} \approx 0.3077$, $p_{13} = 0.5$, $q_{13} \approx 0.3846$, а также координаты $y_1 = 0$, $y_2 = 0.5$, $y_3 = 1$:

   $$
   \frac{\partial C}{\partial y_1} = 4 \left( (0.5 - 0.3077) \cdot 0.3077 \cdot (0 - 0.5) + (0.5 - 0.3846) \cdot 0.3846 \cdot (0 - 1) \right)
   $$
   
   Рассчитываем:

   $$
   = 4 \left( 0.1923 \cdot (-0.5) + 0.1154 \cdot (-1) \right)
   $$
   
   $$
   = 4 \left( -0.09615 - 0.1154 \right) = 4 \cdot (-0.21155) = -0.8462
   $$

   Теперь обновим координату $y_1$:

   $$
   y_1^{(new)} = 0 - \eta \cdot (-0.8462) = 0 + \eta \cdot 0.8462
   $$

   Таким образом, координата $y_1$ сместится в положительном направлении на величину, пропорциональную $\eta$.

2. Для точки $y_2$:
   
   $$
   \frac{\partial C}{\partial y_2} = 4 \left( (p_{12} - q_{12}) q_{12} (y_2 - y_1) + (p_{23} - q_{23}) q_{23} (y_2 - y_3) \right)
   $$
   
   Подставляем значения:

   $$
   = 4 \left( (0.5 - 0.3077) \cdot 0.3077 \cdot (0.5 - 0) + (0.3776 - 0.3077) \cdot 0.3077 \cdot (0.5 - 1) \right)
   $$

   Рассчитываем:

   $$
   = 4 \left( 0.1923 \cdot 0.5 + 0.0699 \cdot (-0.5) \right)
   $$
   
   $$
   = 4 \left( 0.09615 - 0.03495 \right) = 4 \cdot 0.0612 = 0.2448
   $$

   Обновляем координату $y_2$:

   $$
   y_2^{(new)} = 0.5 - \eta \cdot 0.2448
   $$

   Таким образом, координата $y_2$ сместится в отрицательном направлении.

3. Для точки $y_3$:
   
   $$
   \frac{\partial C}{\partial y_3} = 4 \left( (p_{13} - q_{13}) q_{13} (y_3 - y_1) + (p_{23} - q_{23}) q_{23} (y_3 - y_2) \right)
   $$
   
   Подставляем значения:

   $$
   = 4 \left( (0.5 - 0.3846) \cdot 0.3846 \cdot (1 - 0) + (0.3776 - 0.3077) \cdot 0.3077 \cdot (1 - 0.5) \right)
   $$

   Рассчитываем:

   $$
   = 4 \left( 0.1154 \cdot 1 + 0.0699 \cdot 0.5 \right) = 4 \left( 0.1154 + 0.03495 \right) = 4 \cdot 0.15035 = 0.6014
   $$

   Обновляем координату $y_3$:

   $$
   y_3^{(new)} = 1 - \eta \cdot 0.6014
   $$

   Таким образом, координата $y_3$ сместится в отрицательном направлении.

### Шаг 8: Итеративная оптимизация

Процесс продолжается итеративно: на каждом шаге алгоритм пересчитывает вероятности $q_{ij}$ в низкоразмерном пространстве, обновляет координаты точек на основе градиентов, и повторяет шаги до тех пор, пока функция стоимости $C$ не будет минимизирована, то есть пока расхождение между распределениями $p_{ij}$ и $q_{ij}$ не станет минимальным.



##Реализация t-SNE:
Для реализации t-SNE на Python мы можем использовать библиотеку `numpy` для расчета расстояний и вероятностей, а также градиентного спуска для минимизации KL-дивергенции. Мы реализуем основной алгоритм t-SNE шаг за шагом.

### Реализация t-SNE:

```python
import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import pairwise_distances

def calculate_joint_probabilities(X, perplexity=30.0):
    # Вычисляем попарные евклидовы расстояния
    distances = pairwise_distances(X, metric='euclidean', squared=True)
    
    # Определяем сигмы (параметр для гауссиана) через бинарный поиск
    def binary_search_sigma(D_i, target_perplexity):
        beta = 1.0  # начальное значение β
        tol = 1e-5  # допустимая ошибка
        max_iter = 50  # макс. число итераций
        min_beta, max_beta = -np.inf, np.inf
        entropy_diff = lambda P: np.log2(P.sum()) - np.log2(target_perplexity)
        
        for _ in range(max_iter):
            P_i = np.exp(-D_i * beta)
            P_i[i] = 0  # вероятности на себя равны 0
            sum_P_i = np.sum(P_i)
            if sum_P_i == 0:
                sum_P_i = 1e-10  # для избежания деления на ноль
            P_i /= sum_P_i
            entropy = -np.sum(P_i * np.log2(P_i + 1e-10))
            error = entropy - np.log2(target_perplexity)
            
            if np.abs(error) < tol:
                break
            if error > 0:
                min_beta = beta
                if max_beta == np.inf:
                    beta *= 2
                else:
                    beta = (beta + max_beta) / 2
            else:
                max_beta = beta
                beta = (beta + min_beta) / 2
        
        return P_i

    n = X.shape[0]
    P = np.zeros((n, n))
    
    for i in range(n):
        P[i, :] = binary_search_sigma(distances[i], perplexity)

    # Окончательное симметричное распределение
    P = (P + P.T) / (2 * n)
    P = np.maximum(P, 1e-12)  # предотвращение слишком малых значений
    return P

def calculate_q(Y):
    inv_distances = 1 / (1 + pairwise_distances(Y, metric='euclidean', squared=True))
    np.fill_diagonal(inv_distances, 0)
    Q = inv_distances / np.sum(inv_distances)
    return np.maximum(Q, 1e-12)

def tsne(X, perplexity=30.0, learning_rate=200.0, n_iter=1000, n_components=2):
    n, d = X.shape
    Y = np.random.randn(n, n_components) * 1e-4  # начальная низкоразмерная проекция
    P = calculate_joint_probabilities(X, perplexity)
    
    for iter in range(n_iter):
        Q = calculate_q(Y)
        
        # Вычисляем градиенты
        grad = np.zeros_like(Y)
        for i in range(n):
            PQ_diff = (P[i, :] - Q[i, :]) * Q[i, :]
            grad[i] = 4 * np.sum(np.expand_dims(PQ_diff, axis=1) * (Y[i, :] - Y), axis=0)
        
        # Градиентный шаг
        Y -= learning_rate * grad
        
        if iter % 100 == 0:
            cost = np.sum(P * np.log(P / Q))
            print(f"Iteration {iter}: cost = {cost}")
    
    return Y

# Пример данных для работы
X = np.random.randn(100, 50)  # 100 точек в 50-мерном пространстве
Y = tsne(X)
```

### Шаги реализации:
1. **Вычисление вероятностей в высокоразмерном пространстве**: Используется функция `calculate_joint_probabilities`, которая с помощью бинарного поиска определяет сигму для каждой точки для достижения заданной перплексии. Перплексия — это мера неопределенности распределения вероятностей.
   
2. **Вычисление вероятностей в низкоразмерном пространстве**: Используется функция `calculate_q`, которая находит вероятности в новом пространстве с помощью расстояний между точками и их инверсий (t-распределение).

3. **Градиентный спуск**: Основная функция `tsne` выполняет итеративное обновление координат с использованием градиентного спуска для минимизации KL-дивергенции между распределениями $P$ и $Q$.

### Пример работы:
Алгоритм t-SNE будет выводить значение функции стоимости каждые 100 итераций, что позволяет отслеживать процесс минимизации.



